In [11]:
from keras.models import load_model
import argparse
import cv2
import pickle
from imutils import paths
import numpy as np
import os

image_dir = os.path.join((os.getcwd()), "test\\161-11.jpg")
#image_path = (sorted(list(paths.list_images(image_dir))))
#image_dir = "test\161-11.jpg"

#Test image pre-processing
image = cv2.imread(image_dir, cv2.IMREAD_GRAYSCALE)
output = image.copy()

#Face detection from Image
har_path = os.path.join(os.path.dirname(cv2.__file__),"data","haarcascade_frontalface_default.xml")
face_cascade = cv2.CascadeClassifier(har_path)
faces = face_cascade.detectMultiScale(output, 1.3, 5)
x1 = faces[0][0]-int(faces[0][2]/2)
y1 = faces[0][1]-int(faces[0][3]/2)
x2 = faces[0][0]+faces[0][2]+int(faces[0][2]/2)
y2 = faces[0][1]+faces[0][3]+int(faces[0][3]/2)
cropped = image[y1:y2,x1:x2]

image = cv2.resize(cropped, (64,64))
image = image.astype("float")/255.0
image = image.reshape(1, image.shape[0], image.shape[1])  
image = image[:,:,:,np.newaxis]

#Passing image to model for prediction
model = load_model(os.path.join((os.getcwd()), "model\smallvggnet_model"))
lb = pickle.loads(open(os.path.join((os.getcwd()), "model\smallvggnet_LabelBinarizer"), 'rb').read())
preds = model.predict(image)
i = preds.argmax(axis=1)[0]
label = lb.classes_[i]
text = "{}: {:.2f}%".format(label, preds[0][i]*100)
cv2.putText(output, text, (10,30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,255), 2)

#Show output image
cv2.imshow("image", output)
cv2.waitKey(0)

-1